In [ ]:
%reset -f
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sns
import csv
import re
import gc
import sys, os, random

import matplotlib.pyplot as plt # for plotting

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(37)
random.seed(17)

pd.set_option('display.max_rows',1000)

root = '/Users/schwalmdaniel/github/kaggle/home-credit-default-risk'
#root = 'e:/kaggle/home-credit-default-risk'

train=pd.read_csv(root + "/application_train.csv")
test=pd.read_csv(root + "/application_test.csv")
bureau=pd.read_csv(root + "/bureau.csv")
previous_application=pd.read_csv(root + "/previous_application.csv")
installments_payments=pd.read_csv(root + "/installments_payments.csv")
bureau_balance=pd.read_csv(root + "/bureau_balance.csv")
POS_CASH_balance=pd.read_csv(root + "/POS_CASH_balance.csv")
credit_card_balance=pd.read_csv(root + "/credit_card_balance.csv")


# have a look at the ds
#train.head()

def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
POS_CASH_balance.head()

num_aggregations = {
        'CNT_INSTALMENT': ['min', 'max', 'mean','sum','var'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean','sum','var'],
        'SK_DPD': ['min', 'max', 'mean'],
        'SK_DPD_DEF': ['min', 'max', 'mean'],
    }

POS_CASH_balance_cat, cat_cols = one_hot_encoder(POS_CASH_balance, nan_as_category= True)
cat_aggregations = {}
    
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']
    
POS_CASH_balance_agg = POS_CASH_balance_cat.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
POS_CASH_balance_agg.columns = pd.Index(['POSBAL_' + e[0] + "_" + e[1].upper() for e in POS_CASH_balance_agg.columns.tolist()])

train=train.join(POS_CASH_balance_agg,on='SK_ID_CURR', how='left')
test=test.join(POS_CASH_balance_agg,on='SK_ID_CURR', how='left')

del POS_CASH_balance_agg,POS_CASH_balance_cat

In [ ]:
credit_card_balance.head()

num_aggregations = {
        'AMT_BALANCE': ['min', 'max', 'mean'],
        'AMT_CREDIT_LIMIT_ACTUAL': ['min', 'max', 'mean'],
        'AMT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean'],
        'AMT_DRAWINGS_CURRENT': ['min', 'max', 'mean'],
        'AMT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean'],
        'AMT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean'],
        'AMT_INST_MIN_REGULARITY': ['min', 'max', 'mean'],
        'AMT_PAYMENT_CURRENT': ['min', 'max', 'mean'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['min', 'max', 'mean'],
        'AMT_RECEIVABLE_PRINCIPAL': ['min', 'max', 'mean'],
        'AMT_RECIVABLE': ['min', 'max', 'mean'],
        'AMT_TOTAL_RECEIVABLE': ['min', 'max', 'mean'],
        'CNT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean','sum','var'],
        'CNT_DRAWINGS_CURRENT': ['min', 'max', 'mean','sum','var'],
        'CNT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean','sum','var'],
        'CNT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean','sum','var'],
        'CNT_INSTALMENT_MATURE_CUM': ['min', 'max', 'mean','sum','var'],
        'SK_DPD': ['min', 'max', 'mean'],
        'SK_DPD_DEF': ['min', 'max', 'mean'],
    }

credit_card_balance_cat, cat_cols = one_hot_encoder(credit_card_balance, nan_as_category= True)
cat_aggregations = {}
    
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']
    
credit_card_balance_agg = credit_card_balance_cat.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
credit_card_balance_agg.columns = pd.Index(['CREBAL_' + e[0] + "_" + e[1].upper() for e in credit_card_balance_agg.columns.tolist()])

train=train.join(credit_card_balance_agg,on='SK_ID_CURR', how='left')
test=test.join(credit_card_balance_agg,on='SK_ID_CURR', how='left')

del credit_card_balance_agg,credit_card_balance_cat

In [ ]:
prev_app_rejected = previous_application[(previous_application['CODE_REJECT_REASON'] != 'XAP')]\
    .groupby('SK_ID_CURR').CODE_REJECT_REASON.agg(['count']).reset_index()
train=train.merge(prev_app_rejected,on='SK_ID_CURR', how='left')
test=test.merge(prev_app_rejected,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'prev_app_rejected_count'}, inplace=True)
test.rename(columns={'count': 'prev_app_rejected_count'}, inplace=True)
train['prev_app_rejected_count'].fillna(0,inplace=True)
test['prev_app_rejected_count'].fillna(0,inplace=True)

prev_app_applied_diff = previous_application[['SK_ID_CURR','AMT_APPLICATION','AMT_CREDIT']]
prev_app_applied_diff['CREDIT_APPL_DIFF'] =  prev_app_applied_diff['AMT_CREDIT'] - \
    prev_app_applied_diff['AMT_APPLICATION'] 

prev_app_credit_diff = prev_app_applied_diff.groupby('SK_ID_CURR').\
    CREDIT_APPL_DIFF.agg(['min']).reset_index()
train=train.merge(prev_app_credit_diff,on='SK_ID_CURR', how='left')
test=test.merge(prev_app_credit_diff,on='SK_ID_CURR', how='left')
train.rename(columns={'min': 'prev_app_credit_diff_min'}, inplace=True)
                      #,'max': 'prev_app_credit_diff_max','mean': 'prev_app_credit_diff_mean'}
                     
test.rename(columns={'min': 'prev_app_credit_diff_min'}, inplace=True)
                      #,'max': 'prev_app_credit_diff_max','mean': 'prev_app_credit_diff_mean'}
                     #, inplace=True})
train['prev_app_credit_diff_min'].fillna(0,inplace=True)
#train['prev_app_credit_diff_max'].fillna(0,inplace=True)
#train['prev_app_credit_diff_mean'].fillna(0,inplace=True)
test['prev_app_credit_diff_min'].fillna(0,inplace=True)
#test['prev_app_credit_diff_max'].fillna(0,inplace=True)
#test['prev_app_credit_diff_mean'].fillna(0,inplace=True)
    

In [ ]:
num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum','var'],
    }

prev_app_cat, cat_cols = one_hot_encoder(previous_application, nan_as_category= True)
cat_aggregations = {}
    
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']
    
prev_agg = prev_app_cat.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

train=train.join(prev_agg,on='SK_ID_CURR', how='left')
test=test.join(prev_agg,on='SK_ID_CURR', how='left')

del prev_agg

In [ ]:
num_aggregations = {
        'DAYS_PAYMENT_DIFF': ['min', 'max', 'mean','sum'],
        'PAYMENT_DIFF': ['min', 'max', 'mean','sum','var'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum','var'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_RATIO': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    }

ins_pay = installments_payments.drop(['SK_ID_PREV','NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER'],axis=1)
ins_pay['DAYS_PAYMENT_DIFF'] = ins_pay['DAYS_ENTRY_PAYMENT'] - ins_pay['DAYS_INSTALMENT']
ins_pay['PAYMENT_DIFF'] = ins_pay['AMT_INSTALMENT'] - ins_pay['AMT_PAYMENT']
ins_pay['DBD'] = ins_pay['DAYS_INSTALMENT'] - ins_pay['DAYS_ENTRY_PAYMENT']
ins_pay['DAYS_PAYMENT_DIFF'] = ins_pay['DAYS_PAYMENT_DIFF'].apply(lambda x: x if x > 0 else 0)
ins_pay['DBD'] = ins_pay['DBD'].apply(lambda x: x if x > 0 else 0)
ins_pay['PAYMENT_RATIO'] = ins_pay['AMT_PAYMENT'] / ins_pay['AMT_INSTALMENT']

ins_pay_agg = ins_pay.groupby('SK_ID_CURR').agg({**num_aggregations})
ins_pay_agg.columns = pd.Index(['INS_' + e[0] + "_" + e[1].upper() for e in ins_pay_agg.columns.tolist()])

ins_pay_agg.head()
train=train.join(ins_pay_agg,on='SK_ID_CURR', how='left')
test=test.join(ins_pay_agg,on='SK_ID_CURR', how='left')

del ins_pay_agg, ins_pay

In [ ]:
bureaue_active_credit_limit = bureau[(bureau['CREDIT_ACTIVE'] == 'Active') & (bureau['AMT_CREDIT_SUM_LIMIT'].notnull())\
            & (bureau['AMT_CREDIT_SUM_LIMIT'] > 0.0)]\
    .groupby('SK_ID_CURR').AMT_CREDIT_SUM_LIMIT.agg(['sum']).reset_index()

bureau_last_credit_update_active = bureau[(bureau['CREDIT_ACTIVE'] == 'Active')]\
    .groupby('SK_ID_CURR').DAYS_CREDIT_UPDATE.agg(['min','max']).reset_index()
bureau_last_credit_update_closed = bureau[(bureau['CREDIT_ACTIVE'] == 'Closed')]\
    .groupby('SK_ID_CURR').DAYS_CREDIT_UPDATE.agg(['min','max']).reset_index()
bureau_last_credit_update = bureau.groupby('SK_ID_CURR').DAYS_CREDIT_UPDATE.agg(['min','max']).reset_index()

train=train.merge(bureau_last_credit_update_active,on='SK_ID_CURR', how='left')
test=test.merge(bureau_last_credit_update_active,on='SK_ID_CURR', how='left')
train.rename(columns={'max': 'bureau_last_credit_update_active_max',
                      'min': 'bureau_last_credit_update_active_min'}, inplace=True)
test.rename(columns={'max': 'bureau_last_credit_update_active_max',
                     'min': 'bureau_last_credit_update_active_min'}, inplace=True)
train['bureau_last_credit_update_active_max'].fillna(0,inplace=True)
train['bureau_last_credit_update_active_min'].fillna(0,inplace=True)
test['bureau_last_credit_update_active_max'].fillna(0,inplace=True)
test['bureau_last_credit_update_active_min'].fillna(0,inplace=True)

train=train.merge(bureau_last_credit_update_closed,on='SK_ID_CURR', how='left')
test=test.merge(bureau_last_credit_update_closed,on='SK_ID_CURR', how='left')
train.rename(columns={'max': 'bureau_last_credit_update_closed_max',
                      'min': 'bureau_last_credit_update_closed_min'}, inplace=True)
test.rename(columns={'max': 'bureau_last_credit_update_closed_max',
                     'min': 'bureau_last_credit_update_closed_min'}, inplace=True)
train['bureau_last_credit_update_closed_max'].fillna(0,inplace=True)
train['bureau_last_credit_update_closed_min'].fillna(0,inplace=True)
test['bureau_last_credit_update_closed_max'].fillna(0,inplace=True)
test['bureau_last_credit_update_closed_min'].fillna(0,inplace=True)

train=train.merge(bureau_last_credit_update,on='SK_ID_CURR', how='left')
test=test.merge(bureau_last_credit_update,on='SK_ID_CURR', how='left')
train.rename(columns={'max': 'bureau_last_credit_update_max',
                      'min': 'bureau_last_credit_update_min'}, inplace=True)
test.rename(columns={'max': 'bureau_last_credit_update_max',
                     'min': 'bureau_last_credit_update_min'}, inplace=True)
train['bureau_last_credit_update_max'].fillna(0,inplace=True)
train['bureau_last_credit_update_min'].fillna(0,inplace=True)
test['bureau_last_credit_update_max'].fillna(0,inplace=True)
test['bureau_last_credit_update_min'].fillna(0,inplace=True)

In [ ]:
bureau_credit = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_ACTIVE']])

bureau_overdue = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_DAY_OVERDUE']])
bureau_credit_type = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_ACTIVE','CREDIT_TYPE']])
bureau_sum_debt = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_ACTIVE','AMT_CREDIT_SUM_DEBT']])

bureau_credit_active = bureau_credit[bureau_credit['CREDIT_ACTIVE'] == 'Active'].\
    groupby('SK_ID_CURR').CREDIT_ACTIVE.agg(['count']).reset_index()
bureau_credit_closed = bureau_credit[bureau_credit['CREDIT_ACTIVE'] == 'Closed'].\
    groupby('SK_ID_CURR').CREDIT_ACTIVE.agg(['count']).reset_index()
bureau_credit_type_active = bureau_credit_type[bureau_credit_type['CREDIT_ACTIVE'] == 'Active'].\
    groupby('SK_ID_CURR').CREDIT_TYPE.agg(['count']).reset_index()
bureau_credit_type_closed = bureau_credit_type[bureau_credit_type['CREDIT_ACTIVE'] == 'Closed'].\
    groupby('SK_ID_CURR').CREDIT_TYPE.agg(['count']).reset_index()
bureau_sum_debt_active = bureau_sum_debt[bureau_sum_debt['CREDIT_ACTIVE'] == 'Active'].\
    groupby('SK_ID_CURR').AMT_CREDIT_SUM_DEBT.agg(['sum']).reset_index()
bureau_sum_debt_closed = bureau_sum_debt[bureau_sum_debt['CREDIT_ACTIVE'] == 'Closed'].\
    groupby('SK_ID_CURR').AMT_CREDIT_SUM_DEBT.agg(['sum']).reset_index()
bureau_credit_overdue = bureau_overdue.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.agg([
        'min', 
        'max', 
        'mean', 
        'std']).reset_index()
bureau_credit_overdue.fillna(0,inplace=True)
bureau_credit_prolong = pd.DataFrame(bureau[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']])\
    .groupby('SK_ID_CURR').CNT_CREDIT_PROLONG.agg([
        'min', 
        'max', 
        'mean', 
        'std']).reset_index()
bureau_currency_count = pd.DataFrame(bureau[['SK_ID_CURR', 'CREDIT_CURRENCY']])\
    .groupby('SK_ID_CURR').CREDIT_CURRENCY.agg(['count']).reset_index()
bureau_sum_overdue = pd.DataFrame(bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']])\
    .groupby('SK_ID_CURR').AMT_CREDIT_SUM_OVERDUE.agg(['sum']).reset_index()
bureau_sum_overdue.fillna(0,inplace=True)
bureau_max_overdue = pd.DataFrame(bureau[['SK_ID_CURR', 'AMT_CREDIT_MAX_OVERDUE']])\
    .groupby('SK_ID_CURR').AMT_CREDIT_MAX_OVERDUE.agg(['min','max']).reset_index()
bureau_max_overdue.fillna(0,inplace=True)
bureau_credit_active_enddate = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].\
    groupby('SK_ID_CURR').DAYS_CREDIT_ENDDATE.agg(['max','min','mean']).reset_index()
bureau_credit_closed_enddate = bureau[bureau['CREDIT_ACTIVE'] == 'Closed'].\
    groupby('SK_ID_CURR').DAYS_CREDIT_ENDDATE.agg(['max','min','mean']).reset_index()

    
train=train.merge(bureaue_active_credit_limit,on='SK_ID_CURR', how='left')
test=test.merge(bureaue_active_credit_limit,on='SK_ID_CURR', how='left')
train.rename(columns={'sum': 'bureaue_active_credit_limit'}, inplace=True)
test.rename(columns={'sum': 'bureaue_active_credit_limit'}, inplace=True)

train=train.merge(bureau_credit_active_enddate,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_active_enddate,on='SK_ID_CURR', how='left')
train.rename(columns={'max': 'bureau_credit_active_enddate_max','min': 'bureau_credit_active_enddate_min',
                     'mean': 'bureau_credit_active_enddate_mean',}, inplace=True)
test.rename(columns={'max': 'bureau_credit_active_enddate_max','min': 'bureau_credit_active_enddate_min',
                     'mean': 'bureau_credit_active_enddate_mean',}, inplace=True)
train=train.merge(bureau_credit_closed_enddate,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_closed_enddate,on='SK_ID_CURR', how='left')
train.rename(columns={'max': 'bureau_credit_closed_enddate_max','min': 'bureau_credit_closed_enddate_min',
                     'mean': 'bureau_credit_closed_enddate_mean',}, inplace=True)
test.rename(columns={'max': 'bureau_credit_closed_enddate_max','min': 'bureau_credit_closed_enddate_min',
                     'mean': 'bureau_credit_closed_enddate_mean',}, inplace=True)
train['bureau_credit_active_enddate_max'].fillna(0,inplace=True)
train['bureau_credit_active_enddate_min'].fillna(0,inplace=True)
train['bureau_credit_active_enddate_mean'].fillna(0,inplace=True)
train['bureau_credit_closed_enddate_max'].fillna(0,inplace=True)
train['bureau_credit_closed_enddate_min'].fillna(0,inplace=True)
train['bureau_credit_closed_enddate_mean'].fillna(0,inplace=True)
test['bureau_credit_active_enddate_max'].fillna(0,inplace=True)
test['bureau_credit_active_enddate_min'].fillna(0,inplace=True)
test['bureau_credit_active_enddate_mean'].fillna(0,inplace=True)
test['bureau_credit_closed_enddate_max'].fillna(0,inplace=True)
test['bureau_credit_closed_enddate_min'].fillna(0,inplace=True)
test['bureau_credit_closed_enddate_mean'].fillna(0,inplace=True)


train=train.merge(bureau_credit_active,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_active,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_credit_active_count'}, inplace=True)
test.rename(columns={'count': 'bureau_credit_active_count'}, inplace=True)

train=train.merge(bureau_credit_closed,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_closed,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_credit_closed_count'}, inplace=True)
test.rename(columns={'count': 'bureau_credit_closed_count'}, inplace=True)

train=train.merge(bureau_credit_type_active,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_type_active,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_credit_type_active_count'}, inplace=True)
test.rename(columns={'count': 'bureau_credit_type_active_count'}, inplace=True)

train=train.merge(bureau_credit_type_closed,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_type_closed,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_credit_type_closed_count'}, inplace=True)
test.rename(columns={'count': 'bureau_credit_type_closed_count'}, inplace=True)

train=train.merge(bureau_sum_debt_active,on='SK_ID_CURR', how='left')
test=test.merge(bureau_sum_debt_active,on='SK_ID_CURR', how='left')
train.rename(columns={'sum': 'bureau_sum_debt_active_sum'}, inplace=True)
test.rename(columns={'sum': 'bureau_sum_debt_active_sum'}, inplace=True)

train=train.merge(bureau_sum_debt_closed,on='SK_ID_CURR', how='left')
test=test.merge(bureau_sum_debt_closed,on='SK_ID_CURR', how='left')
train.rename(columns={'sum': 'bureau_sum_debt_closed_sum'}, inplace=True)
test.rename(columns={'sum': 'bureau_sum_debt_closed_sum'}, inplace=True)

train=train.merge(bureau_currency_count,on='SK_ID_CURR', how='left')
test=test.merge(bureau_currency_count,on='SK_ID_CURR', how='left')
train.rename(columns={'count': 'bureau_currency_count'}, inplace=True)
test.rename(columns={'count': 'bureau_currency_count'}, inplace=True)

train=train.merge(bureau_sum_overdue,on='SK_ID_CURR', how='left')
test=test.merge(bureau_sum_overdue,on='SK_ID_CURR', how='left')
train.rename(columns={'sum': 'bureau_sum_overdue'}, inplace=True)
test.rename(columns={'sum': 'bureau_sum_overdue'}, inplace=True)

train=train.merge(bureau_max_overdue,on='SK_ID_CURR', how='left')
test=test.merge(bureau_max_overdue,on='SK_ID_CURR', how='left')
train.rename(columns={'max': 'bureau_max_overdue_max','min': 'bureau_max_overdue_min'}, inplace=True)
test.rename(columns={'max': 'bureau_max_overdue_max','min': 'bureau_max_overdue_min'}, inplace=True)

train=train.merge(bureau_credit_prolong,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_prolong,on='SK_ID_CURR', how='left')
train.rename(columns={'min': 'bureau_credit_prolong_min','max': 'bureau_credit_prolong_max',
                      'mean': 'bureau_credit_prolong_mean','std': 'bureau_credit_prolong_std'}, inplace=True)
test.rename(columns={'min': 'bureau_credit_prolong_min','max': 'bureau_credit_prolong_max',
                      'mean': 'bureau_credit_prolong_mean','std': 'bureau_credit_prolong_std'}, inplace=True)

train=train.merge(bureau_credit_overdue,on='SK_ID_CURR', how='left')
test=test.merge(bureau_credit_overdue,on='SK_ID_CURR', how='left')
train.rename(columns={'min': 'bureau_credit_overdue_min','max': 'bureau_credit_overdue_max',
                      'mean': 'bureau_credit_overdue_mean','std': 'bureau_credit_overdue_std'}, inplace=True)
test.rename(columns={'min': 'bureau_credit_overdue_min','max': 'bureau_credit_overdue_max',
                      'mean': 'bureau_credit_overdue_mean','std': 'bureau_credit_overdue_std'}, inplace=True)
train['bureau_credit_overdue_min'].fillna(0,inplace=True)
train['bureau_credit_overdue_max'].fillna(0,inplace=True)
train['bureau_credit_overdue_mean'].fillna(0,inplace=True)
train['bureau_credit_overdue_std'].fillna(0,inplace=True)
train['bureau_credit_prolong_min'].fillna(0,inplace=True)
train['bureau_credit_prolong_max'].fillna(0,inplace=True)
train['bureau_credit_prolong_mean'].fillna(0,inplace=True)
train['bureau_credit_prolong_std'].fillna(0,inplace=True)
train['bureau_credit_active_count'].fillna(0,inplace=True)
train['bureau_credit_closed_count'].fillna(0,inplace=True)
train['bureau_sum_debt_active_sum'].fillna(0,inplace=True)
train['bureau_sum_debt_closed_sum'].fillna(0,inplace=True)
train['bureau_credit_type_active_count'].fillna(0,inplace=True)
train['bureau_credit_type_closed_count'].fillna(0,inplace=True)
train['bureau_currency_count'].fillna(0,inplace=True)
train['bureau_sum_overdue'].fillna(0,inplace=True)
train['bureau_max_overdue_max'].fillna(0,inplace=True)
train['bureau_max_overdue_min'].fillna(0,inplace=True)
train['bureaue_active_credit_limit'].fillna(0,inplace=True)
test['bureau_credit_overdue_min'].fillna(0,inplace=True)
test['bureau_credit_overdue_max'].fillna(0,inplace=True)
test['bureau_credit_overdue_mean'].fillna(0,inplace=True)
test['bureau_credit_overdue_std'].fillna(0,inplace=True)
test['bureau_credit_prolong_min'].fillna(0,inplace=True)
test['bureau_credit_prolong_max'].fillna(0,inplace=True)
test['bureau_credit_prolong_mean'].fillna(0,inplace=True)
test['bureau_credit_prolong_std'].fillna(0,inplace=True)
test['bureau_credit_active_count'].fillna(0,inplace=True)
test['bureau_credit_closed_count'].fillna(0,inplace=True)
test['bureau_sum_debt_active_sum'].fillna(0,inplace=True)
test['bureau_sum_debt_closed_sum'].fillna(0,inplace=True)
test['bureau_credit_type_active_count'].fillna(0,inplace=True)
test['bureau_credit_type_closed_count'].fillna(0,inplace=True)
test['bureau_currency_count'].fillna(0,inplace=True)
test['bureau_sum_overdue'].fillna(0,inplace=True)
test['bureau_max_overdue_max'].fillna(0,inplace=True)
test['bureau_max_overdue_min'].fillna(0,inplace=True)
test['bureaue_active_credit_limit'].fillna(0,inplace=True)

train.head()
                            

In [ ]:
# 'NAME_CONTRACT_TYPE', 2 values, converting to 0/1
train['NAME_CONTRACT_TYPE'] = train['NAME_CONTRACT_TYPE'].apply(lambda x: 0 if x == 'Cash loans' else 1)
test['NAME_CONTRACT_TYPE'] = test['NAME_CONTRACT_TYPE'].apply(lambda x: 0 if x == 'Cash loans' else 1)

In [ ]:
# 'CODE_GENDER', drop XNA as only 4 rows, convert the rest to 0/1
train = train[train['CODE_GENDER'] != 'XNA']
train['CODE_GENDER'] = train['CODE_GENDER'].apply(lambda x: 0 if x == 'F' else 1)
test['CODE_GENDER'] = test['CODE_GENDER'].apply(lambda x: 0 if x == 'F' else 1)

In [ ]:
# FLAG_OWN_CAR
train['FLAG_OWN_CAR'] = train['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'Y' else 0)
test['FLAG_OWN_CAR'] = test['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'Y' else 1)

In [ ]:
# FLAG_OWN_REALTY
train['FLAG_OWN_REALTY'] = train['FLAG_OWN_REALTY'].apply(lambda x: 1 if x == 'Y' else 0)
test['FLAG_OWN_REALTY'] = test['FLAG_OWN_REALTY'].apply(lambda x: 1 if x == 'Y' else 1)

In [ ]:
# where null it should be dropped or mean or average income/annuity

avgAnnuityRate = (train['AMT_ANNUITY']/train['AMT_CREDIT']).mean()
train['AMT_ANNUITY'].fillna(avgAnnuityRate * train['AMT_CREDIT'],inplace=True)
test['AMT_ANNUITY'].fillna(avgAnnuityRate * train['AMT_CREDIT'],inplace=True)


In [ ]:
#  where null mean or average income / goods price
goodsPriceMean = train['AMT_GOODS_PRICE'].mean()
train['AMT_GOODS_PRICE'].fillna(goodsPriceMean,inplace=True)
test['AMT_GOODS_PRICE'].fillna(goodsPriceMean,inplace=True)

In [ ]:
#  categorical, dummify, where null either unknown or most frequent
train['NAME_TYPE_SUITE'].fillna('Unaccompanied',inplace=True)
test['NAME_TYPE_SUITE'].fillna('Unaccompanied',inplace=True)

In [ ]:
# numeric, the older the worse, where null check own car
train['OWN_CAR_AGE'].fillna(100,inplace=True)
test['OWN_CAR_AGE'].fillna(100,inplace=True)
train['OWN_CAR_AGE'] = train['OWN_CAR_AGE'] * -1
test['OWN_CAR_AGE'] = test['OWN_CAR_AGE'] * -1

In [ ]:
# numeric, drop where it is null
train = train[train['CNT_FAM_MEMBERS'] > 0]
test = test[test['CNT_FAM_MEMBERS'] > 0]

In [ ]:
#train.columns.tolist()
train['INCOME_ANNUITY_RATIO'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
test['INCOME_ANNUITY_RATIO'] = test['AMT_ANNUITY'] / test['AMT_INCOME_TOTAL']

train['INCOME_CREDIT_RATIO'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
test['INCOME_CREDIT_RATIO'] = test['AMT_CREDIT'] / test['AMT_INCOME_TOTAL']

train['ANNUITY_CREDIT_RATIO'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
test['ANNUITY_CREDIT_RATIO'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']

train['ANNUITY_GOODS_PRICE_RATIO'] = train['AMT_ANNUITY'] / train['AMT_GOODS_PRICE']
test['ANNUITY_GOODS_PRICE_RATIO'] = test['AMT_ANNUITY'] / test['AMT_GOODS_PRICE']

train['ANNUITY_DAYS_EMPLOYED_RATIO'] = train['AMT_ANNUITY'] / train['DAYS_EMPLOYED']
test['ANNUITY_DAYS_EMPLOYED_RATIO'] = test['AMT_ANNUITY'] / test['DAYS_EMPLOYED']

train['CREDIT_DAYS_EMPLOYED_RATIO'] = train['AMT_CREDIT'] / train['DAYS_EMPLOYED']
test['CREDIT_DAYS_EMPLOYED_RATIO'] = test['AMT_CREDIT'] / test['DAYS_EMPLOYED']

train['GOODS_CREDIT_RATIO'] = train['AMT_CREDIT'] / train['AMT_GOODS_PRICE']
test['GOODS_CREDIT_RATIO'] = test['AMT_CREDIT'] / test['AMT_GOODS_PRICE']

train['INCOME_PER_CAPITA'] = train['AMT_INCOME_TOTAL'] / train['CNT_FAM_MEMBERS']
test['INCOME_PER_CAPITA'] = test['AMT_INCOME_TOTAL'] / test['CNT_FAM_MEMBERS']

train['EXT_SOURCES_MEAN'] = train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
test['EXT_SOURCES_MEAN'] = test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

train['EXT_SOURCES_PROD'] = train['EXT_SOURCE_1'] * train['EXT_SOURCE_2'] * train['EXT_SOURCE_3']
test['EXT_SOURCES_PROD'] = test['EXT_SOURCE_1'] * test['EXT_SOURCE_2'] * test['EXT_SOURCE_3']

train['EXT_SCORES_STD'] = train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
train['EXT_SCORES_STD'] = train['EXT_SCORES_STD'].fillna(train['EXT_SCORES_STD'].mean())
test['EXT_SCORES_STD'] = test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
test['EXT_SCORES_STD'] = test['EXT_SCORES_STD'].fillna(train['EXT_SCORES_STD'].mean())

train['EMPLOY_TO_BIRTH_RATIO'] = train['DAYS_EMPLOYED'] / train['DAYS_BIRTH']
test['EMPLOY_TO_BIRTH_RATIO'] = test['DAYS_EMPLOYED'] / test['DAYS_BIRTH']

train['PHONE_TO_EMPLOY_RATIO'] = train['DAYS_LAST_PHONE_CHANGE'] / train['DAYS_EMPLOYED']
test['PHONE_TO_EMPLOY_RATIO'] = test['DAYS_LAST_PHONE_CHANGE'] / test['DAYS_EMPLOYED']

train['PHONE_TO_BIRTH_RATIO'] = train['DAYS_LAST_PHONE_CHANGE'] / train['DAYS_BIRTH']
test['PHONE_TO_BIRTH_RATIO'] = test['DAYS_LAST_PHONE_CHANGE'] / test['DAYS_BIRTH']

In [ ]:
# fill null with mean for _1, _2, _3
'''mean1 = train['EXT_SOURCE_1'].mean()
mean2 = train['EXT_SOURCE_2'].mean()
mean3 = train['EXT_SOURCE_3'].mean()
train['EXT_SOURCE_1'].fillna(mean1,inplace=True)
train['EXT_SOURCE_2'].fillna(mean2,inplace=True)
train['EXT_SOURCE_3'].fillna(mean3,inplace=True)
test['EXT_SOURCE_1'].fillna(mean1,inplace=True)
test['EXT_SOURCE_2'].fillna(mean2,inplace=True)
test['EXT_SOURCE_3'].fillna(mean3,inplace=True)'''
train['EXT_SOURCE_1'].fillna(0,inplace=True)
train['EXT_SOURCE_2'].fillna(0,inplace=True)
train['EXT_SOURCE_3'].fillna(0,inplace=True)
test['EXT_SOURCE_1'].fillna(0,inplace=True)
test['EXT_SOURCE_2'].fillna(0,inplace=True)
test['EXT_SOURCE_3'].fillna(0,inplace=True)


In [ ]:
# fill with mean all _AVG, _MEDI, _MODE
for col in train.columns.tolist():
    if (col.endswith('_AVG') or col.endswith('_MEDI') or col.endswith('_MODE')) and col not in ['FONDKAPREMONT_MODE','HOUSETYPE_MODE',
                    'WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']: 
        #print (col)
        mean = train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)
        

In [ ]:
# EMERGENCYSTATE_MODE
train['EMERGENCYSTATE_MODE'] = train['EMERGENCYSTATE_MODE'].apply(lambda x: 1 if x == 'Y' else 0)
test['EMERGENCYSTATE_MODE'] = test['EMERGENCYSTATE_MODE'].apply(lambda x: 1 if x == 'Y' else 1)


In [ ]:
#  Fill none with mean or median for all circle
for col in train.columns.tolist():
    if col.endswith('_CIRCLE'):
        mean = train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)


In [ ]:
# negative numeric, drop where it is null
train['DAYS_LAST_PHONE_CHANGE'].fillna(0,inplace=True)
test['DAYS_LAST_PHONE_CHANGE'].fillna(0,inplace=True)


In [ ]:
# all amt_credit req, maybe bin it
train['AMT_REQ_CREDIT_BUREAU_YEAR'].mean()

for col in train.columns.tolist():
    if 'AMT_REQ_CREDIT_BUREAU_' in col:
        #print (col)
        mean = train[col].mean()
        train[col].fillna(mean,inplace=True)
        test[col].fillna(mean,inplace=True)


In [ ]:
gc.collect()

In [ ]:
test.shape

In [ ]:
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset, 
    columns = ['NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
            'OCCUPATION_TYPE','ORGANIZATION_TYPE','FONDKAPREMONT_MODE',
            'HOUSETYPE_MODE','WALLSMATERIAL_MODE'],prefix_sep='__') # ,'WEEKDAY_APPR_PROCESS_START'
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
train.shape

In [ ]:
from sklearn.model_selection import train_test_split

low_correlation_cols = ['ORGANIZATION_TYPE__Trade: type 5','ORGANIZATION_TYPE__Transport: type 2',
                        'NONLIVINGAPARTMENTS_MODE','FLAG_DOCUMENT_12','ORGANIZATION_TYPE__Telecom',
                        'ORGANIZATION_TYPE__Industry: type 6','bureau_credit_prolong_min','ORGANIZATION_TYPE__Housing',
                        'OCCUPATION_TYPE__Realty agents','NAME_HOUSING_TYPE__Co-op apartment','FLAG_DOCUMENT_5',
                        'ORGANIZATION_TYPE__Legal Services','ORGANIZATION_TYPE__Industry: type 7',
                        'ORGANIZATION_TYPE__Advertising','FLAG_DOCUMENT_20',
                        'ORGANIZATION_TYPE__Business Entity Type 1','FLAG_CONT_MOBILE',
                        'NAME_TYPE_SUITE__Group of people','FLAG_MOBIL','WALLSMATERIAL_MODE__Others',
                        'AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_HOUR','HOUSETYPE_MODE__terraced house',
                       'WEEKDAY_APPR_PROCESS_START']

X = train.drop(['SK_ID_CURR','TARGET'] + low_correlation_cols, axis=1)
y = train['TARGET']
X_test = test.drop(['SK_ID_CURR','TARGET'] + low_correlation_cols, axis=1)

#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, RepeatedKFold

import gc
import csv

cnt = 0
p_buf = []
n_splits = 2
n_repeats = 2
kf = RepeatedKFold(
    n_splits=n_splits, 
    n_repeats=n_repeats, 
    random_state=0)
auc_buf = []   

params = {
            'nthread':4,
            'n_estimators':10000,
            'learning_rate':0.02,
            'num_leaves':34,
            'colsample_bytree':0.9497036,
            'subsample':0.8715623,
            'subsample_freq':1,
            'max_depth':8,
            'reg_alpha':0.041545473,
            'reg_lambda':0.0735294,
            'min_split_gain':0.0222415,
            'min_child_weight':39.3259775,
            'random_state':0,
    'feature_fraction': 0.15,
            'verbose':-1,
    'metric': 'auc',
    'boosting_type': 'gbdt',
        'objective': 'binary',
}

'''params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 12,
        'num_leaves': 31,
        'learning_rate': 0.025,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.85,
        'bagging_freq': 5,
        'verbose': 0,
        'num_threads': 8,
        'lambda_l2': 1.5,
        'min_gain_to_split': 0,
    }  
'''
for train_index, valid_index in kf.split(X):
    print('Fold {}/{}'.format(cnt + 1, n_splits))
    
    model = lgb.train(
        params,
        lgb.Dataset(X.loc[train_index], y.loc[train_index], feature_name=X.columns.tolist()),
        num_boost_round=10000,
        valid_sets=[lgb.Dataset(X.loc[valid_index], y.loc[valid_index])],
        early_stopping_rounds=100,
        verbose_eval=100,
    )

    if cnt == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        print(tuples[:200])

    p = model.predict(X.loc[valid_index], num_iteration=model.best_iteration)
    #auc = roc_auc_score(y.loc[valid_index], p)

    #print('{} AUC: {}'.format(cnt, auc))

    p = model.predict(X_test, num_iteration=model.best_iteration)
    if len(p_buf) == 0:
        p_buf = np.array(p)
    else:
        p_buf += np.array(p)
    #auc_buf.append(auc)

    cnt += 1
    if cnt > 0: # Comment this to run several folds
        break
    
    del model
    gc.collect

#auc_mean = np.mean(auc_buf)
#auc_std = np.std(auc_buf)
#print('AUC = {:.6f} +/- {:.6f}'.format(auc_mean, auc_std))

preds = p_buf/cnt

subm = pd.DataFrame()
subm['SK_ID_CURR'] = test['SK_ID_CURR']
subm['TARGET'] = preds
subm.to_csv('home-default-risk_lgbm.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)
subm.head()